# TP4

## Constantes et modules

In [4]:
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation, FFMpegWriter

## FTCS et l’équation d’onde

### Question 3

On veut caractériser les oscillations d'une corde de piano de longueur $L$ fixée aux deux bouts. La corde est frappée par un marteau à une distance $d$ d'une de ses extrémités. La situation est représentée à la figure $\textcolor{red}{\text{N}}$.

<center><img src="Q3_schéma.JPG" style="width: 600px;"/></center>
<center>Figure N. AJOUTER LE TITRE ET LA REF AUX CONSIGNES

$$\textcolor{red}{\text{AJOUTER LA REF ET LE TITER ET LE NO DE FIGURE}}$$

On sait que le comportement de la corde tendue respecte l'équation d'onde en une dimension. L'équation d'onde à une dimension correspond à :
$$\frac{\partial^2\phi}{\partial t^2} = v^2\frac{\partial^2\phi}{\partial x^2}$$
On doit adapter cette équation afin de pouvoir la résoudre numériquement. Pour ce faire, on divise la corde en $N$ petits intervalles de longueur $a$. La dérivée par rapport à $x$ peut alors être approximée par une forme discrète :
$$\frac{\partial^2\phi}{\partial x^2} = \frac{1}{a^2}[\phi(x+a,t) + \phi(x-a, t) - 2\phi(x,t)]$$
En remplaçant dans l'équation d'onde, on obtient l'équation différentielle totale suivante :
$$\frac{\partial^2\phi}{\partial t^2} = \frac{v^2}{a^2}[\phi(x+a,t) + \phi(x-a, t) - 2\phi(x,t)]$$
On veut maintenant convertir l'équation différentielle d'ordre 2 en un système de 2 équations différentielles d'ordre 1. En posant le changement de variable $\psi(x,t) = \frac{\text{d}\phi}{\text{d}t}$, on obtient le système d'équations suivant :
$$\psi(x,t) = \frac{\text{d}\phi}{\text{d}t}$$
$$\frac{\text{d}\psi}{\text{d}t} = \frac{v^2}{a^2}[\phi(x+a,t) + \phi(x-a, t) - 2\phi(x,t)]$$
En utilisant la méthode d'Euler avec un pas temporel $h$, on obtient les équations FTCS suivantes :
$$\phi(x,t+h) = \phi(x, t) + h\psi(x, t)$$
$$\psi(x, t+h) = \psi(x, t) + h\frac{v^2}{a^2}[\phi(x+a,t) + \phi(x-a, t) - 2\phi(x,t)]$$
On peut maintenant déterminer numériquement le comportement de la corde pour les conditions initiales données dans l'énoncé.  On a donc une corde de longueur $L=1$ m, de diamètre $d=10$ cm, avec $v=100\ \text{ms}^{-1}$. Initialement, on a $\phi(x)=0$ et
$$\psi(x) = C\frac{x(L-x)}{L^2}e^{-\frac{(x-d)^2}{2\sigma^2}}$$
où $C=1\ \text{ms}^{-1}$ et $\sigma = 0,3$ m.

On code la fonction *corde_FTCS*. Cette fonction prend en argument les *arrays* *phi_0* [m] et *psi_0* [m/s], qui correspondent respectivement aux conditions initiales $\phi(x)$ et $\psi(x)$, ainsi que la longueur *L* [m] de la corde, le nombre *N* de tranches spatiales, la valeur finale de temps *t_f* [s] qui nous intéresse, le nombre *N_t* de tranches temporelles et la valeur du paramètre *v* [$\text{s}^{-1}$]. La fonction détermine d'abord la valeur des pas temporel *h* et spatial *a* à partir des valeurs de *L*, *N*, *t_f* et *N_t*. La fonction utilise ensuite les équations FTCS déterminées plus haut afin de calculer de manière itérative les valeurs de $\phi$ et de $\psi$ pour chaque valeur de temps $t$. Elle retourne ensuite les valeurs de temps *t* dans un *array numpy* à une dimension et les valeurs de *phi* et de *psi* séparément dans deux *arrays numpy* à 2 dimensions où chaque ligne représente un incrément de temps.

In [5]:
# Méthode FTCS pour déterminer le comportement de la corde
# Arguments:
# phi_0 : array 1D des valeurs initiales de phi [m]
# psi_0 : array 1D des valeurs initiales de psi [m/s]
# L : longueur de la corde [m]
# N : nombre de divisions en x [-]
# t_f : valeur maximale de temps [s]
# N_t : nombre de divisions en t [-]
# v : valeur du paramètre v [s^(-1)]
# Retourne :
# t : array numpy 1D représentant les valeurs de temps [s]
# phi : array numpy 2D des valeurs de phi ou chaque rangée est associée 
#       à une valeur de temps et chaque colonne à une position [m]
# psi : array numpy 2D des valeurs de psi ou chaque rangée est associée 
#       à une valeur de temps et chaque colonne à une position [m/s]

def corde_FTCS(phi_0, psi_0, L, N, t_f, N_t, v):
    # pas temporel et spatial
    h = t_f / N_t
    a = L / N
    
    # variables indépendantes
    t = np.linspace(0, t_f, N_t)
    x = np.linspace(0, L, N)

    # valeurs initiales en listes
    phi = [list(phi_0)]
    psi = [list(psi_0)]

    # processus itératif en listes (zéro au début et à la fin pour les bouts fixes)
    for i in range(len(t)):
        psi_i = [0]
        phi_i = [0]
        for j in range(len(x))[1:-1]:
            phi_j = phi[i][j] + h*psi[i][j]
            psi_j = psi[i][j] + h*v**2/a**2 * (phi[i][j+1] + phi[i][j-1] - 2*phi[i][j])
            phi_i.append(phi_j)
            psi_i.append(psi_j)
        phi_i.append(0)
        psi_i.append(0)
        phi.append(phi_i)
        psi.append(psi_i)
    
    # conversion en array
    return t, np.array(phi), np.array(psi)

On peut maintenant déterminer le comportement de la corde de $t=0$ à $t=100$ ms pour les valeurs de paramètres et les valeurs initiales données.

In [6]:
# données
L = 1
N = 100
t_f = 0.1
h = 10**(-6)
N_t = int(t_f / h)
C = 1
d = 0.1
sigma = 0.3
v = 100

# conditions initiales
init_phi = np.zeros(N)
x = np.linspace(0, L, N)
init_psi = C*x*(L-x) / L**2 * np.exp(-(x-d)**2 / (2*sigma**2))

t, phi, psi = corde_FTCS(init_phi, init_psi, L, N, t_f, N_t, v)

On représente graphiquement la position de la corde au temps $t=100$ ms.

In [7]:
# représentation graphique à t=0.1
fig1 = plt.figure(figsize=(10, 2))
fig1.patch.set_facecolor('white')
plt.xlabel('$x$ [m]')
plt.ylabel('$\phi$ [m]')
plt.plot(x, phi[-1])
plt.title("Position de la corde à $t=100$ ms")
plt.show()

<IPython.core.display.Javascript object>

Ce résultat, qui ne correspond pas à ce à quoi on s'attend physiquement, semble indiquer que la solution donnée par la méthode FTCS s'éloigne de la solution physique lorsque le temps avance. On pourra mieux observer ce comportement à la question 4 grâce à une animation.

### Question 4

On veut maintenant observer le comportement de la corde dans l'intervalle de temps. Pour ce faire, on veut représenter les résultats obtenus à la question 3 dans une animation. Pour ce faire, on utilise la fonction *FuncAnimation* du module *matplotlib.animation*. Afin d'avoir une animation et un temps de calcul tous deux d'une durée raisonnable, on choisit de tracer seulement 1 itération temporelle sur 100. On aura donc 1000 images à générer pour animer la corde de 0 à 100 ms. On sauvegarde l'animation et on la fait apparaître dans le *notebook*.

In [8]:
# création de la figure et la ligne qui contiendra les données
fig2 = plt.figure(figsize=(10,2))
fig2.patch.set_facecolor('white')
ax = plt.axes(xlim=(0, 1), ylim=(-0.001, 0.001))
line, = ax.plot([], [], lw=2)
plt.xlabel("x [m]")
plt.ylabel("$\phi$ [m]")
plt.title("Position de la corde de 0 à 100 ms")

# 1 frame pour 100 itérations temporelles
phi_anim = phi[::200]
psi_anim = psi[::200]

# fonction d'initialisation
def fonc_init():
    line.set_data([], [])
    return line,

# fonction d'animation
def anim(i):
    x = np.linspace(0, 1, 100)
    y = phi_anim[i]
    line.set_data(x, y)
    return line,

# animation
corde1 = FuncAnimation(fig2, anim, init_func=fonc_init, frames=500, interval=10, repeat=True)

# sauvegarde
writervideo = FFMpegWriter(fps=30)
corde1.save("corde1.mp4", writer=writervideo)

plt.show()

<IPython.core.display.Javascript object>

On remarque que le comportement de la corde suit initialement le comportement prévu physiquement. En effet, on observe une déformation qui part du point où le marteau frappe la corde et qui se propage le long de celle-ci en étant réfléchie aux extrémités. Puis, de petites oscillations non-physiques commencent à apparaître. Ces oscillations deviennent de plus en plus grandes jusqu'à ce qu'elles soient de taille comparable au mouvement du début de l'animation. On voit donc très bient que la méthode FTCS donne, dans ce cas-ci, une solution qui diverge de la vraie solution lorsque le temps avance.

### Question 5

Maintenant que nous avons observé le comportement de la corde tel que prédit par la méthode FTCS, on veut en analyser la stabilité. Pour ce faire, on peut utiliser la méthode de von Newmann. La démarche pour déterminer la stabilité de la méthode FTCS appliquée à l'équation d'onde est présentée en détails dans la section 9.3.2 de *Computational Physics - Revised and expanded* par Mark Newman. Tout d'abord, on représente nos 2 équations FTCS en séries de Fourier. On regroupe alors les coefficients de Fourier des deux équations en un vecteur pour chaque terme de la série. On peut maintenant représenter l'évolution du vecteur des coefficients de Fourier $\boldsymbol{c}(t)$ grâce à un opérateur linéaire $A$ de la façon suivante :
$$\boldsymbol{c}(t+h) = A\boldsymbol{c}(t)$$
On peut représenter le vecteur $\boldsymbol{c}(t)$ en fonction des vecteurs propres $v_i$ de l'opérateur $A$ :
$$\boldsymbol{c}(t) = \alpha_1v_1 + \alpha_2v_2$$
où $\alpha_1$ et $\alpha_2$ sont des constantes. On peut maintenant représenter l'équation d'une itération temporelle de la façon suivante :
$$\boldsymbol{c}(t+h) = \alpha_1\lambda_1v_1 + \alpha_2\lambda_2v_2$$
où $\lambda_i$ est la valeur propre de $A$ associée au vecteur propre $v_i$. On remarque donc que les vecteurs propres sont simplement multipliés par la même constante à chaque itération. On aura donc une solution qui ne diverge pas seulement si $|\lambda_1|,|\lambda_2|\leq 1$. Sinon, les vecteurs propres seront multipliés par une valeur plus grande que l'unité à chaque itération et les coefficients vont simplement augmenter jusqu'à tendre vers l'infini. Dans notre cas, les valeurs propres respectent l'expression suivante :
$$|\lambda| = \sqrt{1 + \frac{4h^2v^2}{a^2}\sin^2\frac{ka}{2}}$$
Comme cette expression est toujours plus grande que 1, on voit que la méthode FTCS diverge toujours lorsqu'on l'utilise pour résoudre l'équation d'onde. Ce résultat explique les instabilités observées aux question 3 et 4 qui apparaissent après un certain temps. Si on laissait la simulation continuer pour des valeurs de temps plus élevées, on obtiendrait des instabilités toujours de plus en plus grandes.

### Question 6

Vu l'échec de la méthode FTCS pour résoudre l'équation d'onde, on essaie maintenant la méthode de Crank-Nicolson. Pour l'équation d'onde à une dimension, les équations de la méthode de Crank-Nicolson correspondent à :
$$\phi(x,t+h) - \frac{1}{2}h\psi(x,t+h) = \phi(x,t) + \frac{1}{2}h\psi(x,t)$$
$$\psi(x,t+h) - h\frac{v^2}{2a^2}[\phi(x+a,t+h) + \phi(x-a,t+h) - 2\phi(x,t+h)] = \psi(x,t) + h\frac{v^2}{2a^2}[\phi(x+a,t) + \phi(x-a,t) - 2\phi(x,t)]$$
Comme les équations ne donnent pas explicitement $\phi$ et $\psi$, on doit les déterminer en résolvant le système d'équations linéaires formé par les équations. Pour ce faire, on utilise la fonction *linalg.solve* du module *numpy*, qui résoud des systèmes d'équations linéaires par décomposition LU. On crée donc la méthode *corde_CN*, qui À CONTINUER UNE FOIS QUE TOUT MARCHE

In [57]:
# CRANK-NICOLSON
# À REMPLIR QUAND ÇA MARCHE
x = np.linspace(0, L, N)
print(len(x))

def corde_CN(phi_0, psi_0, L, N, t_f, N_t, v):
    # pas temporel et spatial
    h = t_f / N_t
    a = L / N
    
    # variables indépendantes
    t = np.linspace(0, t_f, N_t)
    x = np.linspace(0, L, N)

    # valeurs initiales en listes
    phi = [list(phi_0)]
    psi = [list(psi_0)]

    # constantes pour simplifier les calculs
    b = h * v**2 / (2*a**2)
    lenx = len(x)

    # processus iteratif
    for i in range(len(t)):
        coeffs = []
        somme = []

        for j in range(lenx)[1:-1]:

            # première équation de C-N
            coeffs_j_1 = [0 for n in range(2*lenx)]
            coeffs_j_1[j] = 1
            coeffs_j_1[lenx + j] = -1/2*h
            somme_j_1 = phi[i][j] + 1/2*h * psi[i][j]

            # bouts fixes
            del(coeffs_j_1[0], coeffs_j_1[lenx-1:lenx+1], coeffs_j_1[-1])

            coeffs.append(coeffs_j_1)
            somme.append(somme_j_1)

            # deuxième équation de C-N
            coeffs_j_2 = [0 for n in range(2*lenx)]
            coeffs_j_2[j-1] = -b
            coeffs_j_2[j] = 2*b
            coeffs_j_2[j+1] = -b
            coeffs_j_2[lenx + j] = 1
            somme_j_2 = psi[i][j] + b * (phi[i][j+1] + phi[i][j-1] - 2*phi[i][j])

            # bouts fixes
            del(coeffs_j_2[0], coeffs_j_2[lenx-1:lenx+1], coeffs_j_2[-1])
            
            coeffs.append(coeffs_j_2)
            somme.append(somme_j_2)

        # formattage pour la résolution
        coeffs_array = np.array(coeffs)
        somme_array = np.array(somme)
        
        
        print(coeffs_array.shape, somme_array.shape)
        
        # résolution du système d'équations
        result = np.linalg.solve(coeffs_array, somme_array)
        print(result)

        phi.append([0] + list(result[:lenx-2]) + [0])
        psi.append([0] + list(result[lenx-2:]) + [0])
    
    return t, phi, psi


100


In [58]:
# données
L = 1
N = 100
t_f = 0.1
h = 10**(-3)
N_t = int(t_f / h)
C = 1
d = 0.1
sigma = 0.3
v = 100

# conditions initiales
init_phi = np.zeros(N)
x = np.linspace(0, L, N)
init_psi = C*x*(L-x) / L**2 * np.exp(-(x-d)**2 / (2*sigma**2))

t, phi, psi = corde_CN(init_phi, init_psi, L, N, t_f, N_t, v)

(196, 196) (196,)
[ 4.77998223e-06  9.36876518e-06  1.35680625e-05  1.71659881e-05
  1.99299921e-05  2.15989531e-05  2.18741033e-05  2.04084138e-05
  1.67940091e-05  1.05470971e-05  1.08980713e-06 -1.22717991e-05
 -3.03744295e-05 -5.42300494e-05 -8.50654618e-05 -1.24370566e-04
 -1.73957229e-04 -2.36031133e-04 -3.13279464e-04 -4.08977963e-04
 -5.27121616e-04 -6.72584215e-04 -8.51313165e-04 -1.07056734e-03
 -1.33920747e-03 -1.66805076e-03 -2.07030376e-03 -2.56209096e-03
 -3.16310021e-03 -3.89737065e-03 -4.79425491e-03 -5.88959374e-03
 -7.22715040e-03 -8.86036189e-03 -1.08544772e-02 -1.32891682e-02
 -1.62617171e-02 -1.98909091e-02 -2.43217842e-02 -2.97314400e-02
 -3.63361167e-02 -4.43998472e-02 -5.42450198e-02 -6.62652745e-02
 -8.09412493e-02 -9.88598071e-02 -1.20737511e-01 -1.47449286e-01
 -1.80063421e-01 -2.19884295e-01 -2.68504561e-01 -3.27868844e-01
 -4.00351533e-01 -4.88851755e-01 -5.96909341e-01 -7.28846418e-01
 -8.89940298e-01 -1.08663457e+00 -1.32679684e+00 -1.62003343e+00
 -1.978

In [52]:
# création de la figure et la ligne qui contiendra les données
fig2 = plt.figure(figsize=(10,2))
fig2.patch.set_facecolor('white')
ax = plt.axes(xlim=(0, 1), ylim=(-0.001, 0.001))
line, = ax.plot([], [], lw=2)
plt.xlabel("x [m]")
plt.ylabel("$\phi$ [m]")
plt.title("Position de la corde de 0 à 100 ms")

# 1 frame pour 100 itérations temporelles
phi_anim = phi
psi_anim = psi

# fonction d'initialisation
def fonc_init():
    line.set_data([], [])
    return line,

# fonction d'animation
def anim(i):
    x = np.linspace(0, 1, 100)
    y = phi_anim[i]
    line.set_data(x, y)
    return line,

# animation
corde1 = FuncAnimation(fig2, anim, init_func=fonc_init, repeat=True)

# sauvegarde
writervideo = FFMpegWriter(fps=30)
corde1.save("corde2.mp4", writer=writervideo)

plt.show()

<IPython.core.display.Javascript object>

In [24]:
test = np.array([[1,2,3],[4,5,6]])

test = np.delete(test, 1, 1)

print(test)

[[1 3]
 [4 6]]
